<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.0.0.2"><span class="toc-item-num">2.0.0.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Лес-деревьев" data-toc-modified-id="Лес-деревьев-2.0.0.3"><span class="toc-item-num">2.0.0.3&nbsp;&nbsp;</span>Лес деревьев</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Логистическая-регрессия-тест" data-toc-modified-id="Логистическая-регрессия-тест-4.0.0.1"><span class="toc-item-num">4.0.0.1&nbsp;&nbsp;</span>Логистическая регрессия тест</a></span></li><li><span><a href="#Дерево-решений-тест" data-toc-modified-id="Дерево-решений-тест-4.0.0.2"><span class="toc-item-num">4.0.0.2&nbsp;&nbsp;</span>Дерево решений тест</a></span></li><li><span><a href="#Лес-деревьев-тест" data-toc-modified-id="Лес-деревьев-тест-4.0.0.3"><span class="toc-item-num">4.0.0.3&nbsp;&nbsp;</span>Лес деревьев тест</a></span></li></ul></li></ul></li><li><span><a href="#Исследование-метрика-AUC-ROC" data-toc-modified-id="Исследование-метрика-AUC-ROC-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Исследование метрика AUC-ROC</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Нужно построить модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)


* 		RowNumber — индекс строки в данных
* 		CustomerId — уникальный идентификатор клиента
* 		Surname — фамилия
* 		CreditScore — кредитный рейтинг
* 		Geography — страна проживания
* 		Gender — пол
* 		Age — возраст
* 		Tenure — сколько лет человек является клиентом банка
* 		Balance — баланс на счёте
* 		NumOfProducts — количество продуктов банка, используемых клиентом
* 		HasCrCard — наличие кредитной карты
* 		IsActiveMember — активность клиента
* 		EstimatedSalary — предполагаемая зарплата

*     Целевой признак:
* 		Exited — факт ухода клиента


## Подготовка данных

In [1]:
! pip install fast_ml
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from fast_ml.model_development import train_valid_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

Посмотрим на данные через info,shape,pairplot
.Проверим данные на пропуски,соответствуют ли названия столбцов нужному виду,посмотрим на то к каким типам относятся данные.

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
df.shape

(10000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
df.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [7]:
pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm')

,0
RowNumber,0.000000
CustomerId,0.000000
Surname,0.000000
CreditScore,0.000000
Geography,0.000000
Gender,0.000000
Age,0.000000
Tenure,9.000000
Balance,0.000000
NumOfProducts,0.000000


Есть небольшое кол-во пропусков в строке Tenure,предлогаю просто удалить их, так как данных не много.

In [8]:
df= df.dropna(subset=['Tenure'])

In [9]:
pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm')

,0
RowNumber,0.000000
CustomerId,0.000000
Surname,0.000000
CreditScore,0.000000
Geography,0.000000
Gender,0.000000
Age,0.000000
Tenure,0.000000
Balance,0.000000
NumOfProducts,0.000000


Прорусков больше нет

Переведем названия столбцов к нужному виду в соответствии snake_case

In [10]:
df.columns = df.columns.str.lower()

In [11]:
df = df.rename(columns={'rownumber': 'row_number', 
                        'customerid': 'customer_id',
                        'creditscore': 'credit_score',
                       'numofproducts':'num_of_products',
                       'hascrcard':'has_cr_card',
                       'isactivemember':'is_active_member',
                       'estimatedsalary':'estimated_salary',
                       })


In [12]:
df

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,15719294,Wood,800,France,Female,29,2.0,0.00,2,0,0,167773.55,0
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1


In [13]:
df['balance'] = df['balance'].astype('int64')
df['estimated_salary'] = df['estimated_salary'].astype('int64')
df['tenure'] = df['tenure'].astype('int64')

In [14]:
df

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2,0,1,1,1,101348,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807,1,0,1,112542,0
2,3,15619304,Onio,502,France,Female,42,8,159660,3,1,0,113931,1
3,4,15701354,Boni,699,France,Female,39,1,0,2,0,0,93826,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510,1,1,1,79084,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,15719294,Wood,800,France,Female,29,2,0,2,0,0,167773,0
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0,2,1,0,96270,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369,1,1,1,101699,0
9997,9998,15584532,Liu,709,France,Female,36,7,0,1,0,1,42085,1


Вывод:
* данные стали более чистыми и читабильными.Есть желание преобразовать категориальные признаки из столбцов geography и gender при помощи прямого кодирования 

## Исследование задачи

Категориальные признаки из столбцоы geography и gender преобразуем при помощи прямого кодирования,так же удалим столбцы,которые нам не нужны, они будут только увеличивать фрейм после кодирования.Еще необходимо масштабировать числовые признаки.

In [15]:
df = df.drop(['row_number','customer_id','surname'],axis=1)

In [16]:
#df_new = pd.get_dummies(df,drop_first=False)

In [17]:
features_train, target_train,features_valid,target_valid, features_test, target_test = train_valid_test_split(df, 
                                                                                                              target = 'exited', train_size=0.6,
                                                                                                              valid_size=0.2, 
                                                                                                              test_size=0.2,
                                                                                                             random_state=12345)

In [18]:
print(features_train.shape)
print(target_train.shape)

(5454, 10)
(5454,)


In [19]:
print(features_valid.shape)
print(target_valid.shape)

(1818, 10)
(1818,)


In [20]:
print(features_test.shape)
print(target_test.shape)

(1819, 10)
(1819,)


###### Кодировка данных 

In [21]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

##### Масштабирование

In [22]:
numeric = ['credit_score', 'age', 'tenure', 'balance','estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

##### Логистическая регрессия 

In [23]:
model = LogisticRegression(random_state=12345)
parametrs = {'max_iter':range(1,1001,100),
            'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
win_pred = grid.predict(features_valid)
print('F1 of Logistic egression',f1_score(win_pred,target_valid))

F1 of Logistic egression 0.375


##### Дерево решений 

Используем GridSearchCV для выявения лучшего значения гиппермапарметров 

In [24]:
model = DecisionTreeClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2)}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
predicted_valid = grid.predict(features_valid)
print('F1 of  DecisionTree',f1_score(predicted_valid,target_valid))


F1 of  DecisionTree 0.5787781350482315


##### Лес деревьев

In [25]:
model = RandomForestClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2),
            'n_estimators':(1,30)}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
predicted_valid = grid.predict(features_valid)
print('F1 of RandomForest',f1_score(predicted_valid,target_valid))

F1 of RandomForest 0.5852842809364549


#### Вывод

Без балансировки данных получилось, что самая лучшая модель обучения это лес деревьев  с значением 0.58

## Борьба с дисбалансом

Используем метод downsample - уменьшаем отрицательный класс

In [26]:
def downsample(features,target,fraction):
    features_null=features[target==0]
    features_one=features[target==1]
    target_null=target[target==0]
    target_one=target[target==1]
    
    downsampling_features=pd.concat([features_null.sample(frac=0.3,random_state=1234)]+[features_one])
    downsampling_target=pd.concat([target_null.sample(frac=0.3,random_state=1234)]+[target_one])
    
    downsampling_features,downsampling_target=shuffle(downsampling_features,downsampling_target,random_state=1234)
    return downsampling_features,downsampling_target

In [27]:
features_balance_down, target_balance_down = downsample(features_train,target_train,fraction=4)

##### Обучение после борьбы с дисбалансом 

In [28]:
model = LogisticRegression(random_state=12345)
parametrs = {'max_iter':range(1,1001,100),
            'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_down, target_balance_down)
win_pred = grid.predict(features_valid)
print('F1 of Logistic egression',f1_score(win_pred,target_valid))

F1 of Logistic egression 0.5146579804560262


In [29]:
model = DecisionTreeClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2)}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_down, target_balance_down)
predicted_valid = grid.predict(features_valid)
print('F1 of DecisionTree',f1_score(predicted_valid,target_valid))


F1 of DecisionTree 0.5788964181994192


In [30]:
model = RandomForestClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2),
            'n_estimators':(1,30)}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_down, target_balance_down)
predicted_valid = grid.predict(features_valid)
print('F1 of  RandomForest',f1_score(predicted_valid,target_valid))

F1 of  RandomForest 0.6028921023359288


Используем метод upsample - увеличиваем меньший класс

In [31]:
def upsample(features, target, repeat): 
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat) 
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle( features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [32]:
features_balance_up,target_balance_up=upsample(features_train,target_train,repeat=4)

In [33]:
model = LogisticRegression(random_state=12345,solver='liblinear')
parametrs = {'max_iter':range(1,1001,100),
            'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_up,target_balance_up)
win_pred = grid.predict(features_valid)
print('F1 of Logistic egression',f1_score(win_pred,target_valid))


F1 of Logistic egression 0.5085972850678734


In [34]:
model = DecisionTreeClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2)}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_up,target_balance_up)
win_pred = grid.predict(features_valid)
print('F1 of DecisionTree',f1_score(win_pred,target_valid))


F1 of DecisionTree 0.46195652173913043


In [35]:
model = RandomForestClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2),
            'n_estimators':(1,30)}

grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_up,target_balance_up)
win_pred = grid.predict(features_valid)
print('F1 of RandomForest',f1_score(win_pred,target_valid))

F1 of RandomForest 0.6165191740412979


## Тестирование модели + AUC-ROC

##### Лес деревьев тест

In [36]:
model = RandomForestClassifier(random_state=12345)
parametrs = {'max_depth': range (1,50, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2),
            'n_estimators':(1,30)}

model = RandomForestClassifier(random_state=12345,class_weight='balanced')
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_balance_up,target_balance_up)
win_pred = grid.predict(features_test)

probabilities_train = grid.predict_proba(features_test)
probabilities_one_train = probabilities_train[:, 1]
auc_roc = roc_auc_score(target_test,probabilities_one_train)

auc_roc = roc_auc_score(target_test,probabilities_one_train)
print('F1 of RandomForest',f1_score(win_pred,target_test))
print('AUC_ROC of RandomForest',auc_roc)

F1 of RandomForest 0.5950155763239876
AUC_ROC of RandomForest 0.8399007894920701


### Вывод

* Лучшей моделью  AUC-ROC является RandomForestClassifier с значением в 0.83
* Лучше моделью для F1 является RandomForestClassifier 0.59